In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats

In [ ]:
wide_info_all = pd.read_csv('/home/mam_jupyter/jupyter_dir/artefact/leads_scoring_model/Python/dev/feature_all.csv', index_col = 0)

In [ ]:
wide_info_others = wide_info_all[wide_info_all[list(wide_info_all.columns[[i[0:2] == 'h_' for i in wide_info_all.columns]])].isna().T.all() == True]
wide_info_others = wide_info_others[['mobile', 'd_deal_flag'] + list(pd.read_csv('remained.csv')['action_origin'])] #drop columns with no lift difference

In [ ]:
#wide_info_others.to_csv('/home/mam_jupyter/jupyter_dir/artefact/leads_scoring_model/Python/dev/wide_info_others.csv')
wide_info_others.shape

### Get Extra Information

In [ ]:
lm_others = pd.read_csv('lift_matrix.csv', delimiter = ',')

In [ ]:
city_lift = lm_others[lm_others['action_origin'] == 'c_city'][['action','1']]
city_lift['city'] = city_lift['action'].str.split('_', expand = True)[2]

### Feature Transformation

##### Binary & Categorical Features

In [ ]:
# convert string to int by replacing
wide_info_others = wide_info_others.replace({'d_cust_type':{60261001:1, 60261002:2},
                                             'c_sex':{'性别为男性':1, '性别为女性':2},
                                             'c_age':{'20岁以下':1, '21-25岁':3, '26-30岁':3, '31-35岁':3, '36-40岁':3, 
                                                      '41-45岁':3, '46-50岁':2, '51-55岁':2, '56-60岁':2, '60岁以上':4},
                                             'c_province':{'贵州省':1, '云南省':1, '四川省':1, '重庆市':1, '福建省':2, '海南省':2, 
                                                           '广西壮族自治区':2, '安徽省':2, '江西省':2, '广东省':2, '上海市':2, 
                                                           '西藏自治区':2, '湖北省':2, '江苏省': 2},
                                             'c_city_level':{'一线城市':3, '新一线城市':2, '二线城市':3, '三线城市':3, 
                                                             '四线城市':1, '五线城市':1},
                                             'd_is_deposit_order':{0:2}})

In [ ]:
# deal with province
wide_info_others['c_province'] = [0 if i is np.nan else 3 if isinstance(i, int) == False else i for i in wide_info_others['c_province']]

# deal with city
city_1 = list(city_lift[city_lift['1'] <= 1]['city'])
city_2 = list(city_lift[(city_lift['1'] > 1) & (city_lift['1'] < 1.5)]['city'])
city_3 = list(city_lift[city_lift['1'] >= 1.5]['city'])
wide_info_others['c_city'] = [1 if i in city_1 else 2 if i in city_2 else 3 if i in city_3 else 0 for i in wide_info_others['c_city']]

In [ ]:
# low cov rate features but want to keep because they show great difference between w/o certain features
binary_features = ['d_fir_sec_visit_diff','d_avg_fir_sec_visit_diff','d_avg_firleads_firvisit_diff','d_trail_count_d30',
                   'd_visit_count_d15','d_firfollow_dealf_diff', 'd_firleads_firvisit_diff', 'd_leads_dtbt_ppt','c_register_time', 
                   'c_last_reach_platform_MG服务号','c_last_reach_platform_MGAPP','c_last_reach_platform_MG官网',
                   'd_leads_dtbt_level_2','d_visit_count_d30']

# binary column
wide_info_others[binary_features] = wide_info_others[binary_features].where(wide_info_others[binary_features].isnull(),1).fillna(0).astype(int)

In [ ]:
# deal with datetime features
datetime_list = ['d_fir_card_time', 'd_fir_leads_time', 'd_fir_visit_time']

for col in datetime_list:
    wide_info_others[col] = [1 if i.isoweekday() >= 6 else 2 if isinstance(i.isoweekday(), int) == True \
                             else 0 for i in pd.to_datetime(wide_info_others[col])]

##### Binned Features

In [ ]:
for col in ['d_followup_d30', 'd_followup_d60', 'd_followup_d90']:
    wide_info_others[col] = [1 if i == 1 else 2 if i >= 2 and i <= 25 else 3 if i >= 26 else 0 for i in wide_info_others[col]] 
    
for col in ['d_leads_dtbt_count', 'd_visit_count_d90']:
    wide_info_others[col] = [1 if i == 1 else 2 if i >= 2 and i <= 3 else 3 if i > 3 else 0 for i in wide_info_others[col]]
    
for col in ['d_firlead_dealf_diff', 'd_lastlead_dealf_diff']:
    wide_info_others[col] = [1 if i <= 30 else 2 if i >= 31 and i <= 180 else 3 if i > 180 else 0 for i in wide_info_others[col]]  
    
wide_info_others['d_followup_d7'] = [1 if i == 2 else 2 if i >= 9 else 3 if i >= 0 == False \
                                     else 0 for i in wide_info_others['d_followup_d7']]

wide_info_others['d_followup_d15'] = [1 if i == 1 else 2 if i >= 2 and i <= 4 else 3 if i >= 5 and i <= 13 else 4 if i > 13\
                                      else 0 for i in wide_info_others['d_followup_d15']]      
    
wide_info_others['d_followup_ttl'] = [i if i >= 1 and i <= 2 else 3 if i >= 3 and i <= 30 else 4 if i >= 31 \
                                     else 0 for i in wide_info_others['d_followup_ttl']]

wide_info_others['d_card_ttl'] = [1 if i == 1 else 2 if i == 3 else 3 if i > 3 else 0 for i in wide_info_others['d_card_ttl']]

wide_info_others['d_visit_dtbt_count'] = [1 if i == 1 else 2 if i > 1 else 0 for i in wide_info_others['d_visit_dtbt_count']]  

wide_info_others['d_last_dfail_dealf_diff'] = [1 if i < 0 else 2 if i == 0 else 3 if i > 0 else 0 \
                                               for i in wide_info_others['d_last_dfail_dealf_diff']]

wide_info_others['d_visit_ttl'] = [1 if i == 1 else 2 if i > 1 and i <= 4 else 3 if i > 4 else 0 for i in wide_info_others['d_visit_ttl']]

wide_info_others['d_fir_sec_leads_diff'] = [1 if i == 0 else 2 if i >= 0 else 0 for i in wide_info_others['d_fir_sec_leads_diff']]

wide_info_others['d_fircard_firvisit_diff'] = [1 if i == 0 else 2 if i >= 1 and i <= 40 else 3 if i > 40 else 0 \
                                               for i in wide_info_others['d_fircard_firvisit_diff']]

wide_info_others['c_leads_source_nums'] = [1 if i == 0 else 2 if i >= 1 and i <= 3 else 3 if i >= 4 \
                                               else 0 for i in wide_info_others['c_leads_source_nums']]

wide_info_others['d_avg_leads_date'] = [1 if i == 0 else 2 if i >= 1 and i <= 7 else 3 if i >= 8 and i <= 60 else 4 if i > 60 \
                                        else 0 for i in wide_info_others['d_avg_leads_date']]

wide_info_others['d_lasttrail_dealf_diff'] = [1 if i == 0 else 2 if i >= 1 and i <= 17 else 3 if i > 17 \
                                               else 0 for i in wide_info_others['d_lasttrail_dealf_diff']]

wide_info_others['d_avg_visit_date'] = [1 if i == 0 else 2 if i >= 1 and i <= 14 else 3 if i > 14 \
                                               else 0 for i in wide_info_others['d_avg_visit_date']]

wide_info_others['d_avg_visit_dtbt_count'] = [1 if i <= 3 else 2 if i > 3 else 0 for i in wide_info_others['d_avg_visit_dtbt_count']]

wide_info_others['d_leads_count'] = [1 if i >= 1 and i <= 3 else 2 if i > 3 else 0 for i in wide_info_others['d_leads_count']]

wide_info_others['d_leads_dtbt_level_1'] = [1 if i >= 0 and i <= 3 else 2 if i > 3 else 0 for i in wide_info_others['d_leads_dtbt_level_1']]

wide_info_others['d_dealf_firvisit_diff'] = [1 if i >= 0 and i <= 20 else 2 if i >= 21 and i <= 160 else 3 if i > 160 \
                                             else 0 for i in wide_info_others['d_dealf_firvisit_diff']]

wide_info_others['d_avg_fircard_firvisit_diff'] = [1 if i <= 0 else 2 if i > 0 and i <= 30 else 3 if i > 30 and i <= 60 else 4 if \
                                                   i > 60 else 0 for i in wide_info_others['d_avg_fircard_firvisit_diff']]

wide_info_others['d_leads_dtbt_coincide'] = [1 if round(i,1) == 0.5 else 2 if round(i,1) == 1 else 0 if i > 0 == False  \
                                             else 3 for i in wide_info_others['d_leads_dtbt_coincide']]

##### Deal with na

In [ ]:
# fillna with 0
wide_info_others[['d_cust_type','c_sex','c_age','c_city_level', 'c_province','d_is_deposit_order']] = wide_info_others[['d_cust_type','c_sex','c_age','c_city_level', 'c_province','d_is_deposit_order']].fillna(0)

In [ ]:
wide_info_others = wide_info_others.drop(['d_trail_book_tll', 'd_activity_ttl', 'd_dealf_lastvisit_diff', 'd_fir_trail', 'd_fir_dealfail_d',
'd_last_dealfail_d','c_last_activity_time'], axis = 1)

In [ ]:
wide_info_others.shape

In [ ]:
wide_info_others.to_csv('features_others.csv', index = False)